In [21]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report as cr
import pickle

warnings.simplefilter(action = 'ignore', category = FutureWarning)
k_folds = KFold(n_splits = 10)

In [22]:
dataset = pd.read_csv("Hotel_dataset.csv")
dataset.drop(dataset.tail(15491).index, inplace = True)
reviews = [w for w in dataset["Review"]]

In [23]:
dataset.loc[dataset.Rating == 1, 'Rating'] = 0
dataset.loc[dataset.Rating == 2, 'Rating'] = 0
dataset.loc[dataset.Rating == 3, 'Rating'] = 0
dataset.loc[dataset.Rating == 4, 'Rating'] = 1
dataset.loc[dataset.Rating == 5, 'Rating'] = 1

In [24]:
pickle_in = open("word2vec_cbow_vectors.pickle", "rb")
word_vectors_CBOW = pickle.load(pickle_in)
del word_vectors_CBOW[-15491:]

In [25]:
def getReviewVector(review):
    return np.mean([word for word in review], axis = 0)

In [26]:
# CBOW
x = []
for i in range(len(word_vectors_CBOW)):
    x.append(getReviewVector(word_vectors_CBOW[i]))

In [27]:
y = pd.get_dummies(dataset["Rating"])
y = y.iloc[:,1].values

In [28]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [29]:
rfc = RandomForestClassifier(n_estimators = 100)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [30]:
score = accuracy_score(y_test, y_pred)
print(score)
print(cr(y_test, y_pred))

0.843
              precision    recall  f1-score   support

           0       0.82      0.62      0.71       305
           1       0.85      0.94      0.89       695

    accuracy                           0.84      1000
   macro avg       0.83      0.78      0.80      1000
weighted avg       0.84      0.84      0.84      1000



In [31]:
# 10-fold cross-validation - CBOW
scores = cross_val_score(rfc, x, y, cv = k_folds)

In [32]:
# 10-fold cross-validation results - CBOW
print("CV scores: ", scores)
print("Avg CV score: ", scores.mean())

CV scores:  [0.83  0.862 0.864 0.876 0.88  0.81  0.818 0.864 0.874 0.912]
Avg CV score:  0.859


In [33]:
pickle_in = open("word2vec_skip-gram_vectors.pickle", "rb")
word_vectors_Skip_Gram = pickle.load(pickle_in)
del word_vectors_Skip_Gram[-15491:]

In [34]:
# Skip-Gram
x = []
for i in range(len(word_vectors_Skip_Gram)):
    x.append(getReviewVector(word_vectors_Skip_Gram[i]))

In [35]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [36]:
rfc = RandomForestClassifier(n_estimators = 100)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [37]:
score = accuracy_score(y_test, y_pred)
print(score)
print(cr(y_test, y_pred))

0.851
              precision    recall  f1-score   support

           0       0.85      0.62      0.72       305
           1       0.85      0.95      0.90       695

    accuracy                           0.85      1000
   macro avg       0.85      0.79      0.81      1000
weighted avg       0.85      0.85      0.84      1000



In [38]:
# 10-fold cross-validation - Skip-Gram
scores = cross_val_score(rfc, x, y, cv = k_folds)

In [39]:
# 10-fold cross-validation results - Skip-Gram
print("CV scores: ", scores)
print("Avg CV score: ", scores.mean())

CV scores:  [0.808 0.87  0.878 0.892 0.886 0.842 0.828 0.876 0.876 0.912]
Avg CV score:  0.8668000000000001
